# Zadanie domowe

W przypadku obrazów w odcieniach szarości pojedynczy piksel z zakresu [0; 255] reprezentowany jest jako 8-bitowa liczba bez znaku.
Pewnym rozszerzeniem analizy sposobu reprezentacji obrazu może być następujący eksperyment.
Załóżmy, że z każdego z 8 bitów możemy stworzyć pojedynczy obraz binarny (ang. _bit-plane slicing_).
Dla obrazka _100zloty.jpg_ (https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/100zloty.jpg) stwórz 8 obrazów, z których każdy powinien zawierać jedną płaszczyznę bitową.
Podpowiedź $-$ warto sprawdzić, jak realizuje się bitowe operacje logiczne.
Zastosowanie takiej dekompozycji obrazu pozwala na analizę ,,ważności'' poszczególnych bitów.
Jest to użyteczne w kwantyzacji, ale także w kompresji.

W drugim etapie zadania proszę spróbować odtworzyć obraz oryginalny z mniejszej liczby obrazów binarnych.
Warto zacząć od dwóch najbardziej znaczących bitów, a później dodawać kolejne.
Należy utworzyć co najmniej trzy wersje zrekonstruowanych obrazów.
Podpowiedź $-$ rekonstrukcja obrazu to mnożenie przez odpowiednią potęgę liczby 2 (przesunięcie bitowe) oraz dodawanie.

In [ ]:
import cv2
import os

import numpy as np
import requests
import matplotlib.pyplot as plt

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/'
fileName = '100zloty.jpg'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
img = cv2.imread(fileName, cv2.IMREAD_COLOR)

plt.imshow(img)
plt.axis('off')
plt.show()

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(img_gray, 'gray')
plt.axis('off')
plt.show()

In [ ]:
def extract_bit_planes(image):
    bit_planes = []
    for i in range(8):
        bit_plane = (image & (1 << i)) >> i
        bit_planes.append(bit_plane * 255) 
    return bit_planes


bit_planes = extract_bit_planes(img)

In [ ]:
for plane in bit_planes:
    plt.imshow(plane)
    plt.axis('off')
    plt.show()

In [ ]:

two_planes = [bit_planes[6], bit_planes[7]]

def reconstruct_image_from_bit_planes(bit_planes, num_planes):
    """
    Rekonstruuje obraz z podanej liczby płaszczyzn bitowych.
    :param bit_planes: Lista płaszczyzn bitowych.
    :param num_planes: Liczba płaszczyzn bitowych do uwzględnienia w rekonstrukcji.
    :return: Zrekonstruowany obraz.
    """
    reconstructed_image = np.zeros_like(bit_planes[0], dtype=np.uint8)
    for i in range(num_planes):
        reconstructed_image += (bit_planes[-(i + 1)] // 255) * (1 << (7 - i))
    return reconstructed_image



for i in range(1, 8):
    reconstructed_image = reconstruct_image_from_bit_planes(bit_planes, i)
    plt.imshow(reconstructed_image)
    plt.axis('off')
    plt.show()


